In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
img_width, img_height = 150, 150

In [4]:
from keras.applications import vgg16

#base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),include_top=False,weights='imagenet')

base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

def addTopOnNet(bottom_model, num_classes):
    top_model = bottom_model.get_layer("block3_pool").output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(64,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(32,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(16,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(num_classes,activation='sigmoid')(top_model)
    return top_model

FC_Head = addTopOnNet(base_model, 1)

model = Model(inputs = base_model.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [5]:
model.load_weights('/content/drive/MyDrive/Final year project/CNN_test/Test_Video/weights/model00000005.h5')

In [6]:
import cv2
import numpy as np
trained_face_data = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def get_face(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_coordinates = trained_face_data.detectMultiScale(gray_img)
    try:
        y, x, h, w = face_coordinates[0]
        return face_coordinates[0], image[x: x + w + 1, y: y + h + 1, :]
    except:
        return False, 0

In [ ]:
from google.colab.patches import cv2_imshow
cap = cv2.VideoCapture('/content/drive/MyDrive/Final year project/CNN_test/Test_Video/merge.mp4')

alpha = 0.1
ewma = -1
# i = 0
# k = 100

RED = np.array([0, 0, 255])
GREEN = np.array([0, 255, 0])

out = None
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output.mp4", codec, fps, (width, height))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        val, face = get_face(frame)
        if type(val) == bool:
            out.write(frame)
            continue
        face = cv2.resize(face, (img_width, img_height))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        t = np.array([face]) / 255.
        curr = model.predict(t)[0][0]
        ewma = curr if ewma == -1 else curr * alpha + (1 - alpha) * ewma
        #print(curr, ewma)
        color = tuple(GREEN * (ewma) + RED * (1 - ewma))
        frame = cv2.rectangle(frame, (val[0], val[1]), (val[0] + val[2], val[1] + val[3]), color, 2)
        frame = cv2.putText(frame, str(ewma)[:4], (0, 30), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, color, 2)
        out.write(frame)
        #cv2_imshow(frame)
        print(curr, ewma)

    else:
        break


cap.release()
cv2.destroyAllWindows()
out.release()